In [1]:
import pandas as pd
from pandas import json_normalize
from tqdm import tqdm
import time
import os
import requests
from bs4 import BeautifulSoup
import json

In [2]:
file_list = os.listdir()

xlsx_list = [xlsx for xlsx in file_list if xlsx.endswith(".xlsx")]

df = pd.read_excel('./{0}'.format(xlsx_list[0]))
df = df.rename(columns={'월별_수출(천$)':'월별'})

df

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
0,201601,1-10,8453665,1108532,699490,267042,421461,566581,857006,404300,139216,147506,128526
1,201601,1-20,22081425,2798444,1713597,1120283,993571,1197717,2302532,1096189,338288,376100,430450
2,201601,1-31,36260420,4590675,2963953,2773742,1684516,1964416,2617192,1871217,552120,597448,696229
3,201602,1-10,8723002,1148994,738513,330477,483390,427479,360863,530185,156696,154552,210700
4,201602,1-20,21967139,2662160,1777030,1234504,1123759,1301971,1512134,1188557,358157,368288,451911
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,202205,1-20,38547511,6608142,3216826,2224359,4433804,1039848,1188052,1258517,976616,708345,700464
230,202205,1-31,61509507,11716953,5155695,3909811,6435808,1491320,1932894,1894433,1799911,1082749,1067522
231,202206,1-10,14997770,3165371,1213913,623804,1830812,495428,387351,495220,286089,271244,215230
232,202206,1-20,31260422,6357257,2575865,1601042,3561079,981320,703353,1033221,559725,572752,462993


In [3]:
"""
if df.tail(1)['기간'].item() != '1-31':
    df_1 = pd.concat([df[(df['기간'] == '1-31')], df.tail(1)])
else:
    df_1 = pd.concat([df[(df['기간'] == '1-31')]])

df_2 = data_pre(df_1).resample(rule='Y').sum()
df_3 = data_pre(df_1).resample(rule='M').sum()

df_2
"""

"\nif df.tail(1)['기간'].item() != '1-31':\n    df_1 = pd.concat([df[(df['기간'] == '1-31')], df.tail(1)])\nelse:\n    df_1 = pd.concat([df[(df['기간'] == '1-31')]])\n\ndf_2 = data_pre(df_1).resample(rule='Y').sum()\ndf_3 = data_pre(df_1).resample(rule='M').sum()\n\ndf_2\n"

In [4]:
def data_pre(df_1):
    df_1['월별'] = pd.to_datetime(df_1['월별'], format="%Y%m")
    df_1 = df_1.set_index('월별')
    return df_1

aa = df.tail(1)['월별'].item()
bb = round(aa, -2)
a = bb-100
b = aa-101
c = aa-100
d = df.tail(1)['기간'].item()

print(aa, bb, a, b, c, d)

202206 202200 202100 202105 202106 1-31


In [5]:
if df.tail(1)['기간'].item() != '1-31':
    df_4 = pd.concat([df[(df['기간'] == '1-31') & df['월별'].between(a, b)], df[(df['기간'] == d) & (df['월별'] == c)]])
else:
    df_4 = pd.concat([df[(df['기간'] == '1-31') & df['월별'].between(a, c)]])
    
df_4

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
182,202101,1-31,48006974,8908101,3635859,3815566,1910154,2384452,3460434,1738840,898908,1063384,591156
185,202102,1-31,44706907,8556779,3539787,3373564,2428358,1845075,1450678,1738988,967756,1016984,563550
188,202103,1-31,53690914,9711666,4153911,4201058,2783163,1912330,2312452,2095282,1193894,1162624,706532
191,202104,1-31,51225991,9516174,4184319,3975318,2974647,2062663,1158623,1953874,1050254,1077834,730147
194,202105,1-31,50725161,10260720,4231339,3322196,3127999,1917326,1314529,1755162,1220535,1026777,746839
197,202106,1-31,54778905,11359169,4563557,3846118,3066694,1930454,1896918,1886444,1386692,1103699,724713


In [9]:
if '1-31' != '1-31':
    df_5 = pd.concat([df[(df['기간'] == '1-31') & df['월별'].between(bb, aa)], df[(df['기간'] == d) & (df['월별'] == c)]])
else:
    df_5 = pd.concat([df[(df['기간'] == '1-31') & df['월별'].between(bb, aa)]])
    
df_5

,월별,기간,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
218,202201,1-31,55457577,11015767,5060251,3924570,3724347,1663892,724661,1974518,1529994,1154352,1177138
221,202202,1-31,54008302,10545061,4663201,3633034,4198826,1664397,1703335,1727889,1629532,1145982,1040642
224,202203,1-31,63780433,13297661,5075736,3783383,5534770,2242741,1454676,2111906,1876376,1312426,1176590
227,202204,1-31,57858183,11001497,5078309,4199277,5184185,1832287,951092,1889161,1807617,1077705,1063313
230,202205,1-31,61509507,11716953,5155695,3909811,6435808,1491320,1932894,1894433,1799911,1082749,1067522
233,202206,1-31,57730562,12585417,4801496,3726669,5517019,1655488,1207971,1836806,1694759,996834,786714


In [7]:
df_6 = pd.concat([df_4, df_5])
df_6 = data_pre(df_6).resample(rule='Y').sum()

df_6

,전체,반도체,철강제품,승용자동차,석유제품,무선통신기기,선박,자동차부품,컴퓨터주변기기,정밀기기,가전제품
월별,,,,,,,,,,,
2021-12-31,303134852,58312609,24308772,22533820,16291015,12052300,11593634,11168590,6718039,6451302,4062937
2022-12-31,408075126,82747773,34636184,26903413,36111974,12205613,9182600,13271519,12032948,7766882,7098633
